## Imports

In [1]:
from IPython.display import Math
from lib3d_mec_ginac import *
from math import pi, e
from warnings import filterwarnings
import pandas as pd
import builtins

In [2]:
filterwarnings('ignore')

In [45]:
pd.set_option("display.max_colwidth", 10000)

def print(x):
    if isinstance(x, Matrix):
        x = pd.DataFrame(unatomize(x)).to_string(index=False, header=False)
    builtins.print(x)

Set gravity value

In [4]:
g = get_param('g')
set_value('g', 9.80665)

Setup a few configuration parameters

In [5]:
 # Integration step
delta_t =             .003
# Assembly init problem solver parameters
geom_eq_init_tol =    1e-10
geom_eq_init_relax =  .1
# Assembly problem solver parameters
geom_eq_tol =         delta_t**2 * 10**-3
geom_eq_relax =       .1
# Equilibrium problem solver parameters
dyn_eq_tol =          1.0e-10
dyn_eq_relax =        .1
# Perturbed dynamic state solver parameters
per_dyn_state_tol =   1e-12
# ...

### Generalized coordinates, velocities and accelerations

In [6]:
theta1, dtheta1, ddtheta1 = new_coord('theta1', -pi/6, 0)
theta2, dtheta2, ddtheta2 = new_coord('theta2', -2*pi/6, 0)
theta3, dtheta3, ddtheta3 = new_coord('theta3', -3*pi/6, 0)

### Kinematical parameters

In [7]:
l1, l2 = new_param('l1', 0.4), new_param('l2', 2.0)
l3, l4 = new_param('l3', 1.2), new_param('l4', 1.6)

### Bases

In [8]:
new_base('Barm1', 'xyz', [0, 1, 0], theta1)
new_base('Barm2', 'xyz', 0, 1, 0, theta2)
new_base('Barm3', 'xyz', rotation_tupla=[0, 1, 0], rotation_angle=theta3);

### Vectors

In [9]:
new_vector('OA', l1, 0, 0, 'Barm1')
new_vector('AB', l2, 0, 0, 'Barm2')
new_vector('BC', [l3, 0, 0], 'Barm3')
new_vector('OO2', values=[l4, 0, 0], base='xyz');

### Points

In [10]:
new_point('A',  'O', 'OA')
new_point('B',  'A', 'AB')
new_point('C',  'B', 'BC')
new_point('O2', 'O', 'OO2');

### Dynamical parameters

In [11]:
m1, m2, m3 = new_param('m1', 1), new_param('m2', 1), new_param('m3', 1);

In [12]:
cg1x, cg1z = new_param('cg1x', 0.2), new_param('cg1z', 0.1)
cg2x, cg2z = new_param('cg2x', 1),   new_param('cg2z', 0.1)
cg3x, cg3z = new_param('cg3x', 0.6), new_param('cg3z', 0.1)

In [13]:
new_vector('OArm1_GArm1', cg1x, 0, cg1z, 'Barm1')
new_vector('OArm2_GArm2', cg2x, 0, cg2z, 'Barm2')
new_vector('OArm3_GArm3', cg3x, 0, cg3z, 'Barm3');

In [14]:
I1yy, I2yy, I3yy = [new_param(name, 1) for name in ('I1yy', 'I2yy', 'I3yy')];

In [15]:
I_Arm1 = new_tensor('Iarm1', base='Barm1')
I_Arm2 = new_tensor('Iarm2', base='Barm2')
I_Arm3 = new_tensor('Iarm3', base='Barm3')
I_Arm1[1, 1], I_Arm2[1, 1], I_Arm3[1, 1] = I1yy, I2yy, I3yy

### Frames

In [16]:
new_frame('FArm1',    'O',  'Barm1')
new_frame('FArm2',    'A',  'Barm2')
new_frame('FArm3',    'B',  'Barm3')
new_frame('Fra_ABS2', 'O2', 'xyz');

### Solids

In [17]:
new_solid('Arm1', 'O', 'Barm1', 'm1', 'OArm1_GArm1', 'Iarm1')
new_solid('Arm2', 'A', 'Barm2', 'm2', 'OArm2_GArm2', 'Iarm2')
new_solid('Arm3', 'B', 'Barm3', 'm3', 'OArm3_GArm3', 'Iarm3');

### Joint unknowns

In [18]:
new_unknown('lambda1')
new_unknown('lambda2');

### Inputs

In [19]:
Fx2, Fz2 = new_input('Fx2'), new_input('Fz2')
Fx3, Fz3 = new_input('Fx3'), new_input('Fz3')
My2, My3 = new_input('My2'), new_input('My3')

new_vector('Fext2', Fx2, 0,   Fz2, 'xyz')
new_vector('Fext3', Fx3, 0,   Fz3, 'xyz')
new_vector('Mext2', 0,   My2, 0,   'xyz')
new_vector('Mext3', 0,   My3, 0,   'xyz');

### Force and momentum

In [20]:
K   = new_param('k',     50)
l2x = new_param('l2x',    1)
l3x = new_param('l3x',  0.5)
l3z = new_param('l3z',  0.1)

new_vector('OArm2_L2',  l2x, 0, 0,   'Barm2')
new_vector('OArm3_L3',  l3x, 0, l3z, 'Barm3')

new_point('OL2', 'A', 'OArm2_L2')
new_point('OL3', 'B', 'OArm3_L3')

OL2_OL3 = position_vector('OL2', 'OL3')
FK = K * OL2_OL3
MK = new_vector('MK_GroundPend1', 0, 0, 0, 'xyz');

### Gravity wrenches

In [21]:
Gravity_Arm1 = gravity_wrench('Arm1')
Gravity_Arm2 = gravity_wrench('Arm2')
Gravity_Arm3 = gravity_wrench('Arm3');

### Inertia wrenches

In [22]:
Inertia_Arm1 = inertia_wrench('Arm1')
Inertia_Arm2 = inertia_wrench('Arm2')
Inertia_Arm3 = inertia_wrench('Arm3');

### Constitutive wrenches

In [23]:
SpringA = new_wrench('SpringA', FK,   MK, 'OL2', 'Arm2', 'Constitutive')
SpringR = new_wrench('SpringR', -FK, -MK, 'OL3', 'Arm3', 'Constitutive');

### External wrenches

In [24]:
FMext2 = new_wrench('FMext2', 'Fext2', 'Mext2', 'A', 'Arm2', 'External')
FMext3 = new_wrench('FMext3', 'Fext3', 'Mext3', 'B', 'Arm3', 'External');

### Wrenches sums

In [25]:
Sum_Wrenches_Arm1 = Inertia_Arm1 + Gravity_Arm1
Sum_Wrenches_Arm2 = Inertia_Arm2 + Gravity_Arm2 + SpringA - FMext2
Sum_Wrenches_Arm3 = Inertia_Arm3 + Gravity_Arm3 - SpringA + FMext3

### Twists

In [26]:
Twist_Arm1, Twist_Arm2, Twist_Arm3 = twist('Arm1'), twist('Arm2'), twist('Arm3')

### Symbols matrices

In [27]:
q, q_aux = get_coords_matrix(), get_aux_coords_matrix()

In [28]:
dq, dq_aux = get_velocities_matrix(), get_aux_velocities_matrix()

In [29]:
ddq, ddq_aux = get_accelerations_matrix(), get_aux_accelerations_matrix()

In [30]:
epsilon = get_unknowns_matrix()
param = get_params_matrix()
input = get_inputs_matrix()

### Kinematic equations

In [31]:
O2C = position_vector('O2', 'C')
e_x = new_vector('e_x', 1, 0, 0, 'xyz')
e_z = new_vector('e_z', 0, 0, 1, 'xyz');

Phi

In [32]:
Phi = Matrix(shape=[2, 1])
Phi[0] = O2C * e_x
Phi[1] = O2C * e_z

In [33]:
print(Phi)

 cos(theta2)*l2+cos(theta1)*l1+cos(theta3)*l3-l4
   -sin(theta1)*l1-sin(theta2)*l2-l3*sin(theta3)


dPHi

In [34]:
dPhi = derivative(Phi)
print(dPhi)

 -dtheta2*sin(theta2)*l2-sin(theta1)*dtheta1*l1-dtheta3*l3*sin(theta3)
 -dtheta1*cos(theta1)*l1-cos(theta3)*dtheta3*l3-dtheta2*cos(theta2)*l2


In [35]:
ddPhi = derivative(dPhi)
print(ddPhi)

 -dtheta2**2*cos(theta2)*l2-sin(theta1)*ddtheta1*l1-cos(theta3)*dtheta3**2*l3-dtheta1**2*cos(theta1)*l1-sin(theta2)*ddtheta2*l2-l3*sin(theta3)*ddtheta3
  dtheta3**2*l3*sin(theta3)+sin(theta1)*dtheta1**2*l1-ddtheta2*cos(theta2)*l2+dtheta2**2*sin(theta2)*l2-cos(theta3)*l3*ddtheta3-cos(theta1)*ddtheta1*l1


Beta

In [36]:
beta = -dPhi
beta = subs(beta, dq, 0)
beta = subs(beta, dq_aux, 0)
print(pd.DataFrame(beta).to_string(index=False, header=False))

 0
 0


Phi_q

In [37]:
Phi_q = jacobian(Phi.transpose(), Matrix.block(2, 1, q, q_aux))
print(Phi_q)

 -sin(theta1)*l1
 -sin(theta2)*l2
 -l3*sin(theta3)
 -cos(theta1)*l1
 -cos(theta2)*l2
 -cos(theta3)*l3


In [38]:
dPhi_dq = jacobian(dPhi.transpose(), Matrix.block(2, 1, dq, dq_aux))
print(dPhi_dq)

 -sin(theta1)*l1
 -sin(theta2)*l2
 -l3*sin(theta3)
 -cos(theta1)*l1
 -cos(theta2)*l2
 -cos(theta3)*l3


In [39]:
gamma = -ddPhi
gamma = subs(gamma, ddq, 0)
gamma = subs(gamma, ddq_aux, 0)
print(gamma)

  dtheta2**2*cos(theta2)*l2+cos(theta3)*dtheta3**2*l3+dtheta1**2*cos(theta1)*l1
 -dtheta3**2*l3*sin(theta3)-sin(theta1)*dtheta1**2*l1-dtheta2**2*sin(theta2)*l2


Phi_init

In [40]:
Phi_init = Matrix([theta1 + pi / 2])
dPhi_init = Matrix([dtheta1 + pi / 2])

### Dynamic equations

In [41]:
Dyn_eq_VP = Matrix(shape=[3, 1], values=[
    Sum_Wrenches_Arm1*diff(Twist_Arm1, dtheta1) + Sum_Wrenches_Arm2*diff(Twist_Arm2,dtheta1) + Sum_Wrenches_Arm3*diff(Twist_Arm3,dtheta1),
    Sum_Wrenches_Arm1*diff(Twist_Arm1, dtheta2) + Sum_Wrenches_Arm2*diff(Twist_Arm2,dtheta2) + Sum_Wrenches_Arm3*diff(Twist_Arm3,dtheta2),
    Sum_Wrenches_Arm1*diff(Twist_Arm1, dtheta3) + Sum_Wrenches_Arm2*diff(Twist_Arm2,dtheta3) + Sum_Wrenches_Arm3*diff(Twist_Arm3,dtheta3)
])
print(Dyn_eq_VP)

                                                                                                                                                                                                                                                                                                                                                                                              cos(theta1)*((sin(theta2)*m2*cg2x-m2*cos(theta2)*cg2z)*dtheta2**2-k*(cos(theta3)*l3z-sin(theta2)*l2-sin(theta3)*l3x+sin(theta2)*l2x)+m2*(sin(theta1)*dtheta1**2*l1-cos(theta1)*ddtheta1*l1)-ddtheta2*(sin(theta2)*m2*cg2z+m2*cos(theta2)*cg2x)+Fz2+m2*g)*l1+(cos(theta2)*l1*(sin(theta2)*cos(theta1)-sin(theta1)*cos(theta2))-sin(theta2)*l1*(sin(theta1)*sin(theta2)+cos(theta1)*cos(theta2)))*(Fx3+(dtheta2*(cos(theta2)*(dtheta2-dtheta1)*l2+dtheta1*cos(theta2)*l2)+sin(theta1)*ddtheta1*l1+dtheta1**2*cos(theta1)*l1+sin(theta2)*ddtheta2*l2)*m3+dtheta3**2*(cos(theta3)*cg3x*m3+sin(theta3)*cg3z*m3)-ddtheta3*(cos(theta3)*cg3z*m3-c

### Output vector

In [42]:
Output = new_matrix('Output', shape=[1, 1])

### Energy equations

In [43]:
Energy = new_matrix('Energy', shape=[1, 1])

Dyn_eq_VP_open = Dyn_eq_VP
Dyn_eq_VP_open = subs(Dyn_eq_VP_open, epsilon, 0)
print(Dyn_eq_VP_open)

                                                                                                                                                                                                                                                                                                                                                                                              cos(theta1)*((sin(theta2)*m2*cg2x-m2*cos(theta2)*cg2z)*dtheta2**2-k*(cos(theta3)*l3z-sin(theta2)*l2-sin(theta3)*l3x+sin(theta2)*l2x)+m2*(sin(theta1)*dtheta1**2*l1-cos(theta1)*ddtheta1*l1)-ddtheta2*(sin(theta2)*m2*cg2z+m2*cos(theta2)*cg2x)+Fz2+m2*g)*l1+(cos(theta2)*l1*(sin(theta2)*cos(theta1)-sin(theta1)*cos(theta2))-sin(theta2)*l1*(sin(theta1)*sin(theta2)+cos(theta1)*cos(theta2)))*(Fx3+(dtheta2*(cos(theta2)*(dtheta2-dtheta1)*l2+dtheta1*cos(theta2)*l2)+sin(theta1)*ddtheta1*l1+dtheta1**2*cos(theta1)*l1+sin(theta2)*ddtheta2*l2)*m3+dtheta3**2*(cos(theta3)*cg3x*m3+sin(theta3)*cg3z*m3)-ddtheta3*(cos(theta3)*cg3z*m3-c

In [44]:
Dyn_eq_L = Dyn_eq_VP
M_qq = jacobian(Dyn_eq_VP_open.transpose(), ddq, True)
print(M_qq)

                                                              -m2*cos(theta1)**2*l1**2-sin(theta1)**2*m2*l1**2-I1yy+sin(theta1)*(cos(theta2)*l1*(sin(theta2)*cos(theta1)-sin(theta1)*cos(theta2))-sin(theta2)*l1*(sin(theta1)*sin(theta2)+cos(theta1)*cos(theta2)))*m3*l1-(cos(theta2)*l1*(sin(theta1)*sin(theta2)+cos(theta1)*cos(theta2))+sin(theta2)*l1*(sin(theta2)*cos(theta1)-sin(theta1)*cos(theta2)))*cos(theta1)*m3*l1
 -(cos(theta2)*l1*(sin(theta1)*sin(theta2)+cos(theta1)*cos(theta2))+sin(theta2)*l1*(sin(theta2)*cos(theta1)-sin(theta1)*cos(theta2)))*cos(theta2)*m3*l2-sin(theta1)*(sin(theta2)*m2*cg2x-m2*cos(theta2)*cg2z)*l1+(cos(theta2)*l1*(sin(theta2)*cos(theta1)-sin(theta1)*cos(theta2))-sin(theta2)*l1*(sin(theta1)*sin(theta2)+cos(theta1)*cos(theta2)))*sin(theta2)*m3*l2-cos(theta1)*(sin(theta2)*m2*cg2z+m2*cos(theta2)*cg2x)*l1
                                                                   -(cos(theta2)*l1*(sin(theta2)*cos(theta1)-sin(theta1)*cos(theta2))-sin(theta2)*l1*(sin(theta1)*sin(th

In [46]:
delta_q = -Dyn_eq_VP_open
delta_q = subs(delta_q, ddq, 0)
delta_q = subs(delta_q, ddq_aux, 0)
print(delta_q)

                                                                                                                                                                                                                                                 -cos(theta1)*l1*((sin(theta2)*m2*cg2x-m2*cos(theta2)*cg2z)*dtheta2**2-k*(cos(theta3)*l3z-sin(theta2)*l2-sin(theta3)*l3x+sin(theta2)*l2x)+sin(theta1)*dtheta1**2*m2*l1+Fz2+m2*g)-(cos(theta2)*l1*(sin(theta2)*cos(theta1)-sin(theta1)*cos(theta2))-sin(theta2)*l1*(sin(theta1)*sin(theta2)+cos(theta1)*cos(theta2)))*(Fx3+dtheta3**2*(cos(theta3)*cg3x*m3+sin(theta3)*cg3z*m3)-k*(cos(theta2)*l2+l3z*sin(theta3)+cos(theta3)*l3x-cos(theta2)*l2x)+m3*(dtheta2*(cos(theta2)*(dtheta2-dtheta1)*l2+dtheta1*cos(theta2)*l2)+dtheta1**2*cos(theta1)*l1))+sin(theta1)*(dtheta1**2*m2*cos(theta1)*l1-Fx2+k*(cos(theta2)*l2+l3z*sin(theta3)+cos(theta3)*l3x-cos(theta2)*l2x)+dtheta2**2*(sin(theta2)*m2*cg2z+m2*cos(theta2)*cg2x))*l1-(cos(theta2)*l1*(sin(theta1)*sin(theta2)+cos(theta1)*cos(t

In [47]:
Phi_init = Matrix.block(2, 1, Phi, Phi_init)
print(Phi_init)

 cos(theta2)*l2+cos(theta1)*l1+cos(theta3)*l3-l4
   -sin(theta1)*l1-sin(theta2)*l2-l3*sin(theta3)
                     1.570796326794896558+theta1
